<a href="https://colab.research.google.com/github/annalamboglia/BigDataHomework/blob/main/Homework3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Installation

In [3]:
# Run below commands in google colab
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.0
!wget -q http://apache.osuosl.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
# unzip it
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
# install findspark 
!pip install -q findspark


Download and install Apache Spark 3.2.1, along with its dependencies (Java Development Kit 8+)

Set Environment Variables to make the Spark runtime visible to Linux OS. Please, note that you can manage multiple versions of spark by pointing to the correct version through environment variables. Run below set of commands to point to Apache Spark 3.2.1 version downloaded earlier.

In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

Test the installation worked successfully.

In [4]:
import findspark
findspark.init()

In [5]:
# Verify the Spark version running on the virtual cluster
from pyspark.context import SparkContext
sc = SparkContext.getOrCreate()

assert  "3." in sc.version, "Verify that the cluster Spark's version is 3.x"

If the environment is not configured properly, the instruction will show the Spark's version that would not be the 3.0.

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession(sc)
print(spark)

In [7]:
#Collegamento al Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Load Yelp Dataset

In [8]:
# Read JSON file into dataframe
path='/content/drive/MyDrive/YELP_DATASET/'

df_review = spark.read.json(path + 'yelp_academic_dataset_review.json')
df_review.printSchema()

df_review  = df_review.drop('_corrupt_record')
df_review.show()

root
 |-- _corrupt_record: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|  3.0|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|
|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|  5.0|I've taken

In [9]:
df_business = spark.read.json(path + 'yelp_academic_dataset_business.json')
df_business.printSchema()
df_business.show()

root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable = true)
 |    |-- GoodForDancing: str

#Per ogni business calcolare la media delle recensioni ed ordinarle in ordine descrescente



In [10]:
from pyspark.sql.functions import col

df_review.groupBy("business_id") \
  .avg("stars") \
  .join(df_business, df_business.business_id==df_review.business_id, 'outer') \
  .select('name','categories',col('avg(stars)').alias("avgStars"),'review_count')\
  .sort(col('avgStars').desc(),col('review_count').desc()) \
  .show()

+--------------------+--------------------+--------+------------+
|                name|          categories|avgStars|review_count|
+--------------------+--------------------+--------+------------+
|            Manolito|Cocktail Bars, Cu...|     5.0|         144|
|Night Lizard Brew...|Food, Breweries, ...|     5.0|         133|
|Aesthetically Spe...|Waxing, Hair Remo...|     5.0|         132|
|Lake Appliance Re...|Appliances & Repa...|     5.0|         123|
|        Diskin Cider|Food, Cideries, T...|     5.0|         117|
|      Low Price Keys|Home Services, Au...|     5.0|         112|
|Walls Jewelry Rep...|Watch Repair, Loc...|     5.0|         112|
|     Luhv Vegan Deli|Restaurants, Vega...|     5.0|         107|
|Aladdin's Market ...|Convenience Store...|     5.0|         105|
|ella & louie flowers|Flowers & Gifts, ...|     5.0|         103|
|Premiere Center f...|Medical Spas, Pro...|     5.0|          97|
|Honky Tonk Party ...|Bus Tours, Event ...|     5.0|          93|
| Faedo Fa

#Trovare i ristoranti migliori a Philadelphia a seconda delle stella


In [ ]:
df_business.select('name','address','categories','city','stars','review_count')\
.where(df_business.city == 'Philadelphia')\
.where(df_business.categories=='Restaurants')\
.sort(df_business.stars.desc(),df_business.review_count.desc())\
.limit(10)\
.show()

+--------------------+--------------------+-----------+------------+-----+------------+
|                name|             address| categories|        city|stars|review_count|
+--------------------+--------------------+-----------+------------+-----+------------+
|         Cheers Cafe|2601 E Westmorela...|Restaurants|Philadelphia|  5.0|           6|
|               Gayle|        617 S 3rd St|Restaurants|Philadelphia|  4.5|          18|
|        Hunan Palace|  6118 Lancaster Ave|Restaurants|Philadelphia|  4.5|           7|
|         Yummy Diner|        34 N 52nd St|Restaurants|Philadelphia|  4.5|           6|
|Shank & Evelyn's ...|       932 S 10th St|Restaurants|Philadelphia|  4.5|           5|
|               Saute|      775 S Front St|Restaurants|Philadelphia|  4.0|          25|
|      Bistro Juliana|2723 E Cumberland St|Restaurants|Philadelphia|  4.0|          14|
|          Roselena's| 1623 E Passyunk Ave|Restaurants|Philadelphia|  4.0|          10|
|        Astral Plane| 1708 Lomb

#Trovare i migliori 10 Business accettano Bitcoin come metodo di pagamento

In [ ]:
df_business.select('name','address','categories','city','stars','review_count')\
.where(df_business.attributes.BusinessAcceptsBitcoin==True)\
.sort(col('stars').desc(),col('review_count').desc())\
.limit(10).show()

+--------------------+--------------------+--------------------+---------------+-----+------------+
|                name|             address|          categories|           city|stars|review_count|
+--------------------+--------------------+--------------------+---------------+-----+------------+
|A New Twist Ballo...|                    |Event Planning & ...|          Boise|  5.0|         276|
|3D Carpet Cleanin...|6610 W Linebaugh ...|Home & Garden, Lo...|          Tampa|  5.0|         203|
|          Oat Bakery|        5 W Haley St|Bakeries, Vegan, ...|  Santa Barbara|  5.0|         123|
|     Blinds Brothers|                    |Home & Garden, Sh...|   Philadelphia|  5.0|          97|
|        Paradise Spa|       174 107th Ave|Hydrotherapy, Mas...|Treasure Island|  5.0|          74|
|            DILIGENT|    263 Riverside Dr|Real Estate, Home...|      Nashville|  5.0|          67|
|          Dj Hecktik|                    |Photo Booth Renta...|  Santa Barbara|  5.0|          67|


#Recensioni Annuali

In [ ]:
from pyspark.sql.functions import *

df_reviews_data=df_review.select(col("date"), 
     year(col("date")).alias("year"), 
     month(col("date")).alias("month"))\
     .groupBy("year") \
     .count() \
     .sort(col('year').desc())

df_reviews_data.show()

+----+------+
|year| count|
+----+------+
|2022| 25257|
|2021|493939|
|2020|442892|
|2019|724905|
|2018|756670|
|2017|697776|
|2016|647161|
|2015|590263|
|2014|446222|
|2013|328691|
|2012|244979|
|2011|198099|
|2010|118081|
|2009| 63595|
|2008| 41149|
|2007| 13107|
|2006|  3452|
|2005|   757|
|null|     1|
+----+------+



In [ ]:
pandasDF = df_reviews_data.toPandas()

In [ ]:
pandasDF.dropna(inplace=True)

#change datatype of column
pandasDF = pandasDF.astype({'year': int})

In [ ]:
pandasDF.to_excel("/content/drive/MyDrive/YELP_DATASET/Recensioni_Anno.xlsx")


#Recensioni Mensili

In [11]:
from pyspark.sql.functions import *

df_reviews_data=df_review.select(col("date"), 
     year(col("date")).alias("year"), 
     month(col("date")).alias("month"))\
     .groupBy("year",'month') \
     .count() \
     .sort(col('year').desc(),col('month').desc()) 

df_reviews_data.show()

+----+-----+-----+
|year|month|count|
+----+-----+-----+
|2022|    1|25257|
|2021|   12|38884|
|2021|   11|36103|
|2021|   10|39380|
|2021|    9|37163|
|2021|    8|44989|
|2021|    7|48392|
|2021|    6|46314|
|2021|    5|45953|
|2021|    4|42971|
|2021|    3|43130|
|2021|    2|34289|
|2021|    1|36371|
|2020|   12|34181|
|2020|   11|33315|
|2020|   10|39095|
|2020|    9|38411|
|2020|    8|40214|
|2020|    7|38294|
|2020|    6|32711|
+----+-----+-----+
only showing top 20 rows



In [12]:
df_reviews_data=df_reviews_data.na.drop()


In [ ]:
pandasDF.to_excel("/content/drive/MyDrive/YELP_DATASET/Recensioni_Anno_mese.xlsx")


#Ristoranti

In [ ]:
from pyspark.sql.functions import col

df_review_dataset=df_review.groupBy("business_id") \
  .avg("stars") \
  .join(df_business, df_business.business_id==df_review.business_id, 'outer') \
  .where(df_business.categories=='Restaurants')\
  .select('is_open','latitude', 'longitude',col('avg(stars)').alias("avgStars"),'review_count')

df_review_dataset.show()

+-------+-------------+--------------+------------------+------------+
|is_open|     latitude|     longitude|          avgStars|review_count|
+-------+-------------+--------------+------------------+------------+
|      0|   27.7693413|   -82.6355183|             3.875|           8|
|      0|28.0116750414|-82.7883693274|               4.0|          12|
|      1|   39.9407892|   -75.2596751|2.5714285714285716|           6|
|      1|   39.5362876|   -119.818231| 3.658536585365854|          37|
|      0|    29.959323|   -90.1720183|3.3333333333333335|           5|
|      1|    38.703965|    -90.398387|               5.0|           8|
|      1|   39.9517142|   -75.1557198|3.5714285714285716|           7|
|      0|    39.929374|   -75.1638426|               3.9|          10|
|      0|   39.9467686|   -74.9669128|             2.625|           8|
|      1|   38.5863902|   -90.2768608|1.0909090909090908|          11|
|      1|   40.0795109|    -75.173133|3.3636363636363638|          19|
|     

#Linear Regressor

In [13]:
from pyspark.ml.feature import VectorAssembler,StringIndexer

columns=df_reviews_data.columns
columns.remove('count')
assembler=VectorAssembler(inputCols=columns, outputCol='features')
df=assembler.transform(df_reviews_data)

stringIndexer=StringIndexer(inputCol='count',outputCol='label')
si_model=stringIndexer.fit(df)
df=si_model.transform(df)
df.show()


+----+-----+-----+-------------+-----+
|year|month|count|     features|label|
+----+-----+-----+-------------+-----+
|2022|    1|25257| [2022.0,1.0]| 54.0|
|2021|   12|38884|[2021.0,12.0]| 95.0|
|2021|   11|36103|[2021.0,11.0]| 77.0|
|2021|   10|39380|[2021.0,10.0]| 97.0|
|2021|    9|37163| [2021.0,9.0]| 84.0|
|2021|    8|44989| [2021.0,8.0]|111.0|
|2021|    7|48392| [2021.0,7.0]|124.0|
|2021|    6|46314| [2021.0,6.0]|116.0|
|2021|    5|45953| [2021.0,5.0]|113.0|
|2021|    4|42971| [2021.0,4.0]|106.0|
|2021|    3|43130| [2021.0,3.0]|108.0|
|2021|    2|34289| [2021.0,2.0]| 74.0|
|2021|    1|36371| [2021.0,1.0]| 79.0|
|2020|   12|34181|[2020.0,12.0]| 73.0|
|2020|   11|33315|[2020.0,11.0]| 70.0|
|2020|   10|39095|[2020.0,10.0]| 96.0|
|2020|    9|38411| [2020.0,9.0]| 92.0|
|2020|    8|40214| [2020.0,8.0]| 99.0|
|2020|    7|38294| [2020.0,7.0]| 91.0|
|2020|    6|32711| [2020.0,6.0]| 69.0|
+----+-----+-----+-------------+-----+
only showing top 20 rows



In [ ]:
df.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- count: long (nullable = false)
 |-- features: vector (nullable = true)
 |-- label: double (nullable = false)



In [14]:
trainingData=df.sampleBy('label', fractions={0:0.7,1:0.7,2: 0.7}, seed=100)
testData=df.subtract(trainingData)


In [19]:
from pyspark.ml.regression import LinearRegression
regressor = LinearRegression(featuresCol = 'features', labelCol = 'label')


In [20]:
#Learn to fit the model from training set
regressor = regressor.fit(trainingData)

#To predict the prices on testing set
pred = regressor.evaluate(testData)

#Predict the model
pred.predictions.show()

/content/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+----+-----+-----+-------------+-----+--------------------+
|year|month|count|     features|label|          prediction|
+----+-----+-----+-------------+-----+--------------------+
|2022|    1|25257| [2022.0,1.0]| 54.0| 0.16666665811447956|
|2021|   12|38884|[2021.0,12.0]| 95.0| -3.3333333432642576|
|2021|   11|36103|[2021.0,11.0]| 77.0| -3.0000000097503516|
|2021|   10|39380|[2021.0,10.0]| 97.0| -2.6666666762365026|
|2021|    9|37163| [2021.0,9.0]| 84.0| -2.3333333427225966|
|2021|    8|44989| [2021.0,8.0]|111.0| -2.0000000092086907|
|2021|    7|48392| [2021.0,7.0]|124.0| -1.6666666756948416|
|2021|    6|46314| [2021.0,6.0]|116.0| -1.3333333421809357|
|2021|    5|45953| [2021.0,5.0]|113.0| -1.0000000086670298|
|2021|    4|42971| [2021.0,4.0]|106.0| -0.6666666751531807|
|2021|    3|43130| [2021.0,3.0]|108.0|-0.33333334163927475|
|2021|    2|34289| [2021.0,2.0]| 74.0|-8.12536882222048E-9|
|2021|    1|36371| [2021.0,1.0]| 79.0| 0.33333332538848026|
|2020|   12|34181|[2020.0,12.0]| 73.0|  

In [22]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")

# Root Mean Square Error
rmse = eval.evaluate(pred.predictions)
print("RMSE: %.3f" % rmse)

# Mean Square Error
mse = eval.evaluate(pred.predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval.evaluate(pred.predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(pred.predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)

/content/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


RMSE: 117.611
MSE: 13832.394
MAE: 102.201
r2: -3.106


#RandomForest Regressor

In [23]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler


columns=df_reviews_data.columns
columns.remove('count')

assembler=VectorAssembler(inputCols=columns,outputCol="features")
df=assembler.transform(df_reviews_data)

stringIndexer=StringIndexer(inputCol='count',outputCol="label")
si_model=stringIndexer.fit(df)
df=si_model.transform(df)

In [26]:
#Split training and testing data
trainingData = df.sampleBy("label", fractions={0:0.7,1:0.7,2:0.7},seed=100)
testData=df.subtract(trainingData)

In [32]:
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor(featuresCol = 'features', labelCol = 'label', numTrees=100, maxDepth=2)
model = rf.fit(trainingData)
test_dt = model.transform(testData)
test_dt.show(truncate=False)

+----+-----+-----+-------------+-----+------------------+
|year|month|count|features     |label|prediction        |
+----+-----+-----+-------------+-----+------------------+
|2022|1    |25257|[2022.0,1.0] |54.0 |1.4009166666666666|
|2021|12   |38884|[2021.0,12.0]|95.0 |0.7869166666666668|
|2021|11   |36103|[2021.0,11.0]|77.0 |0.7869166666666668|
|2021|10   |39380|[2021.0,10.0]|97.0 |0.7869166666666668|
|2021|9    |37163|[2021.0,9.0] |84.0 |0.7869166666666668|
|2021|8    |44989|[2021.0,8.0] |111.0|0.9809166666666667|
|2021|7    |48392|[2021.0,7.0] |124.0|0.9809166666666667|
|2021|6    |46314|[2021.0,6.0] |116.0|0.9809166666666667|
|2021|5    |45953|[2021.0,5.0] |113.0|0.9809166666666667|
|2021|4    |42971|[2021.0,4.0] |106.0|1.4009166666666666|
|2021|3    |43130|[2021.0,3.0] |108.0|1.4009166666666666|
|2021|2    |34289|[2021.0,2.0] |74.0 |1.4009166666666666|
|2021|1    |36371|[2021.0,1.0] |79.0 |1.4009166666666666|
|2020|12   |34181|[2020.0,12.0]|73.0 |0.7869166666666668|
|2020|11   |33

In [ ]:
#coefficient of the regression model
coeff = rf.coefficients

#X and Y intercept
intr = rf.intercept

print ("The coefficient of the model is : %a" %coeff)
print ("The Intercept of the model is : %f" %intr)

In [33]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")

# Root Mean Square Error
rmse = evaluator.evaluate(test_dt,{evaluator.metricName: "rmse"})
print("RMSE: %.3f" % rmse)

# Mean Square Error
mse = evaluator.evaluate(test_dt,{evaluator.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = evaluator.evaluate(test_dt,{evaluator.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = evaluator.evaluate(test_dt,{evaluator.metricName: "r2"})
print("r2: %.3f" %r2)





RMSE: 116.448
MSE: 13560.047
MAE: 100.970
r2: -3.025
